In [ ]:
from scipy import stats

stock_returns = [0.065, 0.0265, -0.0593, -0.001, 0.0346]
mkt_returns = [0.055, -0.09, -0.041, 0.045, 0.022]

beta, intercept, r_value, p_value, std_err = \
    stats.linregress(stock_returns, mkt_returns)

In [ ]:
print (beta,intercept)

In [ ]:
""" Least squares regression with statsmodels """
import numpy as np
import statsmodels.api as sm

# Generate some sample data
num_periods = 9
all_values = np.array([np.random.random(8)
                       for i in range(num_periods)])
#print(all_values)

# Filter the data
y_values = all_values[:, 0]  # First column values as Y
x_values = all_values[:, 1:]  # All other values as X
x_values = sm.add_constant(x_values)  # Include the intercept
results = sm.OLS(y_values, x_values).fit()  # Regress and fit the model
print(results.summary())

In [ ]:
""" A simple linear optimization problem with 2 variables """
import pulp

x = pulp.LpVariable("x", lowBound=0)
y = pulp.LpVariable("y", lowBound=0)

problem = pulp.LpProblem("A simple maximization objective",
                         pulp.LpMaximize)
problem += 3*x + 2*y, "The objective function"
problem += 2*x + y <= 100, "1st constraint"
problem += x + y <= 80, "2nd constraint"
problem += x <= 40, "3rd constraint"
problem.solve()

In [ ]:
for variable  in problem.variables():
    print(variable.name,'=',variable.varValue)

In [ ]:
import pulp

dealers = ["X", "Y", "Z"]
variable_costs = {"X": 500,
                  "Y": 350,
                  "Z": 450}
fixed_costs = {"X": 4000,
               "Y": 2000,
               "Z": 6000}

# Define PuLP variables to solve
quantities = pulp.LpVariable.dicts("quantity", 
                                   dealers, 
                                   lowBound=0, 
                                   cat=pulp.LpInteger) 
is_orders = pulp.LpVariable.dicts("orders", 
                                  dealers, 
                                  cat=pulp.LpBinary)

In [ ]:
"""
This is an example of implementing an IP model
with binary variables the correct way.
"""
# Initialize the model with constraints
model = pulp.LpProblem("A cost minimization problem",
                       pulp.LpMinimize)
model += sum([variable_costs[i]*quantities[i] +
              fixed_costs[i]*is_orders[i] for i in dealers]), \
         "Minimize portfolio cost"
model += sum([quantities[i] for i in dealers]) == 150, \
         "Total contracts required"
model += is_orders["X"]*30 <= quantities["X"] <= \
         is_orders["X"]*100, "Boundary of total volume of X"
model += is_orders["Y"]*30 <= quantities["Y"] <= \
         is_orders["Y"]*90, "Boundary of total volume of Y"
model += is_orders["Z"]*30 <= quantities["Z"] <= \
         is_orders["Z"]*70, "Boundary of total volume of Z"
model.solve()

for variable  in model.variables():
    print(variable.name,'=',variable.varValue)

In [ ]:
""" Linear algebra with NumPy matrices """
import numpy as np

A = np.array([[2, 1, 1],
              [1, 3, 2],
              [1, 0, 0]])
B = np.array([4, 5, 6])

print (np.linalg.solve(A, B ))

In [ ]:
""" LU decomposition with SciPy """
import scipy.linalg as linalg
import numpy as np

A = np.array([[2., 1., 1.],
              [1., 3., 2.],
              [1., 0., 0.]])
B = np.array([4., 5., 6.])

LU = linalg.lu_factor(A)
P, L, U = linalg.lu(A)
x = linalg.lu_solve(LU, B)
print(P)
print(L)
print(U)
print(x)

In [ ]:
""" Cholesky decomposition with NumPy """
import numpy as np

A = np.array([[10., -1., 2., 0.],
              [-1., 11., -1., 3.],
              [2., -1., 10., -1.],
              [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])

L = np.linalg.cholesky(A)
print(L)

In [ ]:
print(np.dot(L,L.T.conj()))

In [ ]:
y = np.linalg.solve(L, B)  # L.L*.x=B; When L*.x=y, then L.y=B

In [ ]:
x = np.linalg.solve(L.T.conj(), y)  # x=L*'.y
print(x)

In [ ]:
print (np.mat(A) * np.mat(x).T)  # B=Ax

In [ ]:
""" QR decomposition with scipy """
import scipy.linalg as linalg
import numpy as np

A = np.array([
    [2., 1., 1.],
    [1., 3., 2.],
    [1., 0., 0]])
B = np.array([4., 5., 6.])

Q, R = linalg.qr(A)  # QR decomposition
y = np.dot(Q.T, B)  # Let y=Q'.B
x = linalg.solve(R, y)  # Solve Rx=y

In [ ]:
print(x)

In [ ]:
""" Solve Ax=B with the Jacobi method """
import numpy as np


def jacobi(A, B, n, tol=1e-10):
    # Initializes x with zeroes with same shape and type as B
    x = np.zeros_like(B)
    
    for it_count in range(n):
        x_new = np.zeros_like(x)        
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (B[i] - s1 - s2) / A[i, i]

        if np.allclose(x, x_new, tol):
            break

        x = x_new

    return x

In [ ]:
A = np.array([[10., -1., 2., 0.],
              [-1., 11., -1., 3.],
              [2., -1., 10., -1.],
              [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 25

In [ ]:
x = jacobi(A, B, n)
print('x = ',x)

In [ ]:
""" Solve Ax=B with the Gauss-Seidel method """
import numpy as np


def gauss(A, B, n, tol=1e-10):
    L = np.tril(A)  # Returns the lower triangular matrix of A
    U = A - L  # Decompose A = L + U
    L_inv = np.linalg.inv(L)
    x = np.zeros_like(B)
    
    for i in range(n):        
        Ux = np.dot(U, x)
        x_new = np.dot(L_inv, B - Ux)
        
        if np.allclose(x, x_new, tol):
            break
            
        x = x_new
        
    return x 

In [ ]:
A = np.array([[10., -1., 2., 0.],
              [-1., 11., -1., 3.],
              [2., -1., 10., -1.],
              [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 100
x = gauss(A, B, n)
print('x = ',x)

In [ ]:
'''
Python code:
Incremental search method
'''
""" An incremental search algorithm """
import numpy as np
 
 
def incremental_search(f, a, b, dx):
    """
    :param f: The function to solve
    :param a: The left boundary x-axis value
    :param b: The right boundary x-axis value
    :param dx: The incremental value in searching
    :return: The x-axis value of the root,
                number of iterations used
    """
    fa = f(a)    
    c = a + dx 
    fc = f(c)    
    n = 1
 
    while np.sign(fa) == np.sign(fc):
        if a >= b:
            return a - dx, n
        
        a = c
        fa = fc
        c = a + dx
        fc = f(c)
        n += 1
 
    if fa == 0:
        return a, n
    elif fc == 0:
        return c, n
    else:
        return (a + c)/2., n

In [ ]:
y = lambda x: x**3 + 2.0*x**2 - 5.
root, iterations = incremental_search(y,-5.,5.,0.001)
print('Root is:', root)
print('Iterations:',iterations)

In [ ]:
""" The bisection method """


def bisection(f, a, b, tol=0.1, maxiter=10):
    """
    :param f: The function to solve
    :param a: The x-axis value where f(a)<0
    :param b: The x-axis value where f(b)>0
    :param tol: The precision of the solution
    :param maxiter: Maximum number of iterations
    :return: The x-axis value of the root,
                number of iterations used
    """
    c = (a+b)*0.5  # Declare c as the midpoint ab
    n = 1  # Start with 1 iteration
    while n <= maxiter:
        c = (a+b)*0.5
        if f(c) == 0 or abs(a-b)*0.5 < tol:
            # Root is found or is very close
            return c, n

        n += 1
        if f(c) < 0:
            a = c
        else:
            b = c
                
    return c, n

In [ ]:
y = lambda x: x**3 + 2*x**2 - 5
root, iterations = bisection(y,-5,5,0.00001,100)
print('Root is:', root)
print('Iterations:',iterations)

In [ ]:
""" The Newton-Raphson method """


def newton(f, df, x, tol=0.001, maxiter=100):
    """
    :param f: The function to solve
    :param df: The derivative function of f
    :param x: Initial guess value of x
    :param tol: The precision of the solution
    :param maxiter: Maximum number of iterations
    :return: The x-axis value of the root,
                number of iterations used
    """
    n = 1

    while n <= maxiter:
        x1 = x - f(x)/df(x)
        if abs(x1 - x) < tol:  # Root is very close
            return x1, n
        else:
            x = x1
            n += 1

    return None, n 

In [ ]:
y = lambda x: x**3 + 2*x**2 - 5

In [ ]:
dy = lambda x: 3*x**2 + 4*x

In [ ]:
root, iterations = newton(y, dy, 5.0, 0.00001, 100)

In [ ]:
print ("Root is:", root)

In [ ]:
print ("Iterations:", iterations)

In [ ]:
""" The secant root-finding method """


def secant(f, a, b, tol=0.001, maxiter=100):
    """
    :param f: The function to solve
    :param a: Initial x-axis guess value
    :param b: Initial x-axis guess value, where b>a
    :param tol: The precision of the solution
    :param maxiter: Maximum number of iterations
    :return: The x-axis value of the root,
                number of iterations used
    """
    n = 1
    while n <= maxiter:
        c = b - f(b)*((b-a)/(f(b)-f(a)))
        if abs(c-b) < tol:
            return c, n

        a = b
        b = c
        n += 1

    return None, n

In [ ]:
y = lambda x: x**3 + 2*x**2 - 5
root, iterations = secant(y,-5,5,0.00001,100)
print('Root is:', root)
print('Iterations:',iterations)

In [ ]:
"""
Documentation at
http://docs.scipy.org/doc/scipy/reference/optimize.html
"""
import scipy.optimize as optimize

y = lambda x: x**3 + 2.*x**2 - 5.
dy = lambda x: 3.*x**2 + 4.*x
# Call method: bisect(f, a, b[, args, xtol, rtol, maxiter, ...])
print ("Bisection method: {}" \
      .format(optimize.bisect(y, -5., 5., xtol=0.00001)))

# Call method: newton(func, x0[, fprime, args, tol, ...])
print ("Newton's method: {}" \
      .format(optimize.newton(y, 5., fprime=dy)))
# When fprime=None, then the secant method is used.
print ("Secant method: {}" \
      .format(optimize.newton(y, 5.)))

# Call method: brentq(f, a, b[, args, xtol, rtol, maxiter, ...])
print ("Brent's method:{}" \
      .format(optimize.brentq(y, -5., 5.))) 

In [ ]:
""" Price a European option by the binomial tree model """
#from StockOption import StockOption
import math
import numpy as np

class StockOption(object):

    def __init__(self, S0, K, r, T, N, params):
        self.S0 = S0
        self.K = K
        self.r = r
        self.T = T
        self.N = max(1, N) # Ensure N have at least 1 time step
        self.STs = None  # Declare the stock prices tree
        
        """ Optional parameters used by derived classes """
        self.pu = params.get("pu", 0)  # Probability of up state
        self.pd = params.get("pd", 0)  # Probability of down state
        self.div = params.get("div", 0)  # Dividend yield
        self.sigma = params.get("sigma", 0)  # Volatility
        self.is_call = params.get("is_call", True)  # Call or put
        self.is_european = params.get("is_eu", True)  # Eu or Am

        """ Computed values """
        self.dt = T/float(N)  # Single time step, in years
        self.df = math.exp(
            -(r-self.div) * self.dt)  # Discount factor

In [ ]:
class BinomialEuropeanOption(StockOption):

    def __setup_parameters__(self):
        """ Required calculations for the model """
        self.M = self.N + 1  # Number of terminal nodes of tree
        self.u = 1 + self.pu  # Expected value in the up state
        self.d = 1 - self.pd  # Expected value in the down state
        self.qu = (math.exp((self.r-self.div)*self.dt) -
                   self.d) / (self.u-self.d)
        self.qd = 1-self.qu

    def _initialize_stock_price_tree_(self):
        # Initialize terminal price nodes to zeros
        self.STs = np.zeros(self.M)

        # Calculate expected stock prices for each node
        for i in range(self.M):
            self.STs[i] = self.S0*(self.u**(self.N-i))*(self.d**i)

    def _initialize_payoffs_tree_(self):
        # Get payoffs when the option expires at terminal nodes
        payoffs = np.maximum(
            0, (self.STs-self.K) if self.is_call
            else(self.K-self.STs))

        return payoffs

    def _traverse_tree_(self, payoffs):
        # Starting from the time the option expires, traverse
        # backwards and calculate discounted payoffs at each node
        for i in range(self.N):
            print(payoffs)
            print(self.qu)
            print(payoffs[:-1] * self.qu)
            print(payoffs[1:] * self.qu)
            payoffs = (payoffs[:-1] * self.qu +
                       payoffs[1:] * self.qd) * self.df
            #print(payoffs)

        return payoffs

    def __begin_tree_traversal__(self):
        payoffs = self._initialize_payoffs_tree_()
        return self._traverse_tree_(payoffs)

    def price(self):
        """ The pricing implementation """
        self.__setup_parameters__()
        self._initialize_stock_price_tree_()
        payoffs = self.__begin_tree_traversal__()
        return payoffs[0]  # Option value converges to first node

In [ ]:
eu_option = BinomialEuropeanOption(50,50,0.05,0.5,2,{"pu":0.2,"pd":0.2,"is_call":False})
print(eu_option.price())

In [ ]:
""" Price a European or American option by the binomial tree """
#from StockOption import StockOption
import math
import numpy as np


class BinomialTreeOption(StockOption):

    def _setup_parameters_(self):
        self.u = 1 + self.pu  # Expected value in the up state
        self.d = 1 - self.pd  # Expected value in the down state
        self.qu = (math.exp((self.r-self.div)*self.dt) -
                   self.d)/(self.u-self.d)
        self.qd = 1-self.qu

    def _initialize_stock_price_tree_(self):
        # Initialize a 2D tree at T=0
        self.STs = [np.array([self.S0])]

        # Simulate the possible stock prices path
        for i in range(self.N):
            prev_branches = self.STs[-1]
            st = np.concatenate((prev_branches*self.u,
                                 [prev_branches[-1]*self.d]))
            self.STs.append(st)  # Add nodes at each time step

    def _initialize_payoffs_tree_(self):
        # The payoffs when option expires
        return np.maximum(
            0, (self.STs[self.N]-self.K) if self.is_call
            else (self.K-self.STs[self.N]))

    def __check_early_exercise__(self, payoffs, node):
        early_ex_payoff = \
            (self.STs[node] - self.K) if self.is_call \
            else (self.K - self.STs[node])

        return np.maximum(payoffs, early_ex_payoff)

    def _traverse_tree_(self, payoffs):
        for i in reversed(range(self.N)):
            # The payoffs from NOT exercising the option
            payoffs = (payoffs[:-1] * self.qu +
                       payoffs[1:] * self.qd) * self.df

            # Payoffs from exercising, for American options
            if not self.is_european:
                payoffs = self.__check_early_exercise__(payoffs, 
                                                        i)
        return payoffs

    def __begin_tree_traversal__(self):
        payoffs = self._initialize_payoffs_tree_()
        return self._traverse_tree_(payoffs)

    def price(self):
        self._setup_parameters_()
        self._initialize_stock_price_tree_()
        payoffs = self.__begin_tree_traversal__()

        return payoffs[0]

In [ ]:
am_option = BinomialTreeOption(50,50,0.05,0.5,2,{"pu":0.2,"pd":0.2,"is_call":False,"is_eu":False})
am_option.price()

In [ ]:
""" Price an option by the binomial CRR model """
import math


class BinomialCRROption(BinomialTreeOption):

    def _setup_parameters_(self):
        self.u = math.exp(self.sigma * math.sqrt(self.dt))
        self.d = 1./self.u
        self.qu = (math.exp((self.r-self.div)*self.dt) -
                   self.d)/(self.u-self.d)
        self.qd = 1-self.qu


In [ ]:
eu_option = BinomialCRROption(50,50,0.05,0.5,2,{'sigma':0.3,'is_call':False})
print('European put: %s' % eu_option.price())
am_opton = BinomialCRROption(50,50,0.05,0.5,2,{'sigma':0.3,'is_call':False,'is_eu':False})
print('American put: %s' % am_option.price())

In [ ]:
class BinomialLROption(BinomialTreeOption):

    def _setup_parameters_(self):
        odd_N = self.N if (self.N%2 == 1) else (self.N+1)
        d1 = (math.log(self.S0/self.K) +
              ((self.r-self.div) +
               (self.sigma**2)/2.) *
              self.T) / (self.sigma * math.sqrt(self.T))
        d2 = (math.log(self.S0/self.K) +
              ((self.r-self.div) -
               (self.sigma**2)/2.) *
              self.T) / (self.sigma * math.sqrt(self.T))
        pp_2_inversion = \
            lambda z, n: \
            .5 + math.copysign(1, z) * \
            math.sqrt(.25 - .25 * math.exp(
                -((z/(n+1./3.+.1/(n+1)))**2.)*(n+1./6.)))
        pbar = pp_2_inversion(d1, odd_N)

        self.p = pp_2_inversion(d2, odd_N)
        self.u = 1/self.df * pbar/self.p
        self.d = (1/self.df - self.p*self.u)/(1-self.p)
        self.qu = self.p
        self.qd = 1-self.p

In [ ]:
eu_option = BinomialLROption(50,50,0.05,0.5,3,{'sigma':0.3,'is_call':False})
print('European put: %s' % eu_option.price())
am_opton = BinomialLROption(50,50,0.05,0.5,3,{'sigma':0.3,'is_call':False,'is_eu':False})
print('American put: %s' % am_option.price())

In [ ]:
class BinomialLRWithGreeks(BinomialLROption):

    def __new_stock_price_tree__(self):
        """
        Create additional layer of nodes to our
        original stock price tree
        """
        self.STs = [np.array([self.S0*self.u/self.d,
                              self.S0,
                              self.S0*self.d/self.u])]

        for i in range(self.N):
            prev_branches = self.STs[-1]
            st = np.concatenate((prev_branches * self.u,
                                 [prev_branches[-1] * self.d]))
            self.STs.append(st)

    def price(self):
        self._setup_parameters_()
        self.__new_stock_price_tree__()
        payoffs = self.__begin_tree_traversal__()

        """ Option value is now in the middle node at t=0"""
        option_value = payoffs[len(payoffs)/2]

        payoff_up = payoffs[0]
        payoff_down = payoffs[-1]
        S_up = self.STs[0][0]
        S_down = self.STs[0][-1]
        dS_up = S_up - self.S0
        dS_down = self.S0 - S_down

        """ Get delta value """
        dS = S_up - S_down
        dV = payoff_up - payoff_down
        delta = dV/dS

        """ Get gamma value """
        gamma = ((payoff_up-option_value)/dS_up -
                 (option_value-payoff_down)/dS_down) / \
                ((self.S0+S_up)/2. - (self.S0+S_down)/2.)

        return option_value, delta, gamma

In [ ]:
eu_call = BinomialLRWithGreeks(50, 50, 0.05, 0.5, 300, {"sigma": 0.3, "is_call": True})
results = eu_call.price()
print ("European put values")
print ('Price: %s\nDelta: %s\nGamma: %s' % results)
eu_put = BinomialLRWithGreeks(50, 50, 0.05, 0.5, 300, {"sigma":0.3, "is_call": False})
results = eu_put.price()
print ("European put values")
print ("Price: %s\nDelta: %s\nGamma: %s" % results)

In [ ]:

class TrinomialTreeOption(BinomialTreeOption):

    def _setup_parameters_(self):
        """ Required calculations for the model """
        self.u = math.exp(self.sigma*math.sqrt(2.*self.dt))
        self.d = 1/self.u
        self.m = 1
        self.qu = ((math.exp((self.r-self.div) *
                             self.dt/2.) -
                    math.exp(-self.sigma *
                             math.sqrt(self.dt/2.))) /
                   (math.exp(self.sigma *
                             math.sqrt(self.dt/2.)) -
                    math.exp(-self.sigma *
                             math.sqrt(self.dt/2.))))**2
        self.qd = ((math.exp(self.sigma *
                             math.sqrt(self.dt/2.)) -
                    math.exp((self.r-self.div) *
                             self.dt/2.)) /
                   (math.exp(self.sigma *
                             math.sqrt(self.dt/2.)) -
                    math.exp(-self.sigma *
                             math.sqrt(self.dt/2.))))**2.

        self.qm = 1 - self.qu - self.qd

    def _initialize_stock_price_tree_(self):
        """ Initialize a 2D tree at t=0 """
        self.STs = [np.array([self.S0])]

        for i in range(self.N):
            prev_nodes = self.STs[-1]
            self.ST = np.concatenate(
                (prev_nodes*self.u, [prev_nodes[-1]*self.m,
                                     prev_nodes[-1]*self.d]))
            self.STs.append(self.ST)

    def _traverse_tree_(self, payoffs):
        """ Traverse the tree backwards """
        for i in reversed(range(self.N)):
            payoffs = (payoffs[:-2] * self.qu +
                       payoffs[1:-1] * self.qm +
                       payoffs[2:] * self.qd) * self.df

            if not self.is_european:
                payoffs = self.__check_early_exercise__(payoffs,
                                                        i)

        return payoffs

In [ ]:
eu_put = TrinomialTreeOption(50, 50, 0.05, 0.5, 2,{"sigma": 0.3, "is_call": False})
print ("European put: %s" % eu_put.price())
am_option = TrinomialTreeOption(50, 50, 0.05, 0.5, 2,{"sigma": 0.3, "is_call": False, "is_eu": False})
print ("American put: %s" % am_option.price())


In [ ]:
class ImpliedVolatilityModel(object):

    def __init__(self, S0, r, T, div, N,
                 is_call=False):
        self.S0 = S0
        self.r = r
        self.T = T
        self.div = div
        self.N = N
        self.is_call = is_call

    def _option_valuation_(self, K, sigma):
        # Use the binomial Leisen-Reimer tree
        lr_option = BinomialLROption(
            self.S0, K, self.r,  self.T, self.N,
            {"sigma": sigma,
             "is_call": self.is_call,
             "div": self.div})
        return lr_option.price()

    def get_implied_volatilities(self, Ks, opt_prices):
        impvols = []
        for i in range(len(Ks)):
            # Bind f(sigma) for use by the bisection method
            f = lambda sigma: \
                self._option_valuation_(
                    Ks[i], sigma) - opt_prices[i]
            impv = bisection(f, 0.01, 0.99, 0.0001, 100)[0]
            impvols.append(impv)
        return impvols

In [ ]:
strikes = [ 75, 80, 85, 90, 92.5, 95, 97.5,100, 105, 110, 115, 120, 125]
put_prices = [0.16, 0.32, 0.6, 1.22, 1.77, 2.54, 3.55,4.8, 7.75, 11.8, 15.96, 20.75, 25.81]
model = ImpliedVolatilityModel(99.62, 0.0248, 78/365.,0.0182, 77, is_call=False)
impvols_put = model.get_implied_volatilities(strikes, put_prices) 

In [ ]:
>>> import matplotlib.pyplot as plt
>>> plt.plot(strikes, impvols_put)
>>> plt.xlabel('Strike Prices')
>>> plt.ylabel('Implied Volatilities')
>>> plt.title('AAPL Put Implied Volatilities expiring in 78 days')
>>> plt.show()

In [ ]:
import oandapy
account_id= '101-001-6722193-001'
key='696b275e1ccb748a410af5b1d36eead9-506c9671ff4b0a0dabec0d829e7620fd'
oanda = oandapy.API(environment="practice", access_token=key)
response = oanda.get_prices(instruments="EUR_USD")
print (response)

In [ ]:
>>> prices = response["prices"]
>>> bidding_price = float(prices[0]["bid"])
>>> asking_price = float(prices[0]["ask"])
>>> instrument = prices[0]["instrument"]
>>> time = prices[0]["time"]

>>> print ("[%s] %s bid=%s ask=%s" % (
>>>     time, instrument, bidding_price, asking_price))

In [ ]:
from datetime import datetime, timedelta
# set the trade to expire after one day
trade_expire = datetime.now() + timedelta(days=1)
trade_expire = trade_expire.isoformat("T") + "Z"
trade_expire



In [14]:
# get a list of trades
from oandapyV20 import API
import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.orders as orders
import json

order = {
  "order": {
    "price": "1.2",
    "stopLossOnFill": {
      "timeInForce": "GTC",
      "price": "1.22"
    },
    "timeInForce": "GTC",
    "instrument": "EUR_USD",
    "units": "100",
    "type": "LIMIT",
    "positionFill": "DEFAULT"
        
  }
}
market_order = {
  "order": {
    "instrument": "EUR_USD",
    "units": "100",
    "type": "MARKET",
    "positionFill": "DEFAULT"
        
  }
}

accountID = "101-001-6722193-001"
client = API(access_token="04e86c6b0c7bfe71bf7f98d2b1a6496d-dee13f6f97c4508b949c42deaf95925b")
r = orders.OrderCreate(accountID, market_order)
client.request(r)
print(r.response)

{'relatedTransactionIDs': ['69', '70'], 'orderFillTransaction': {'id': '70', 'financing': '0.0000', 'accountID': '101-001-6722193-001', 'accountBalance': '99939.1173', 'batchID': '69', 'instrument': 'EUR_USD', 'orderID': '69', 'tradeOpened': {'units': '100', 'tradeID': '70'}, 'requestID': '24331496182447387', 'userID': 6722193, 'pl': '0.0000', 'price': '1.19500', 'type': 'ORDER_FILL', 'reason': 'MARKET_ORDER', 'commission': '0.0000', 'time': '2017-09-22T20:50:51.266870049Z', 'units': '100', 'fullPrice': {'asks': [{'price': '1.19500', 'liquidity': '10000000'}], 'bids': [{'price': '1.19475', 'liquidity': '10000000'}], 'closeoutBid': '1.19460', 'timestamp': '2017-09-22T20:50:51.203872045Z', 'closeoutAsk': '1.19515'}}, 'lastTransactionID': '70', 'orderCreateTransaction': {'id': '69', 'accountID': '101-001-6722193-001', 'timeInForce': 'FOK', 'instrument': 'EUR_USD', 'positionFill': 'DEFAULT', 'requestID': '24331496182447387', 'userID': 6722193, 'reason': 'CLIENT_ORDER', 'batchID': '69', 'un

In [ ]:
"""
Implementing the trend-following algorithm
for trading foreign currencies
"""
import oandapy
from datetime import datetime
import pandas as pd


class ForexSystem(oandapy.Streamer):
    def __init__(self, *args, **kwargs):
        oandapy.Streamer.__init__(self, *args, **kwargs)
        self.oanda = oandapy.API(kwargs["environment"],
                                 kwargs["access_token"])

        self.instrument = None
        self.account_id = None
        self.qty = 0
        self.resample_interval = '10s'
        self.mean_period_short = 5
        self.mean_period_long = 20
        self.buy_threshold = 1.0
        self.sell_threshold = 1.0

        self.prices = pd.DataFrame()
        self.beta = 0
        self.is_position_opened = False
        self.opening_price = 0
        self.executed_price = 0
        self.unrealized_pnl = 0
        self.realized_pnl = 0
        self.position = 0
        self.dt_format = "%Y-%m-%dT%H:%M:%S.%fZ"
        
    def begin(self, **params):
        self.instrument = params["instruments"]
        self.account_id = params["accountId"]
        self.qty = params["qty"]
        self.resample_interval = params["resample_interval"]
        self.mean_period_short = params["mean_period_short"]
        self.mean_period_long = params["mean_period_long"]
        self.buy_threshold = params["buy_threshold"]
        self.sell_threshold = params["sell_threshold"]

        self.start(**params)  # Start streaming prices
        
    def on_success(self, data):
        time, symbol, bid, ask = self.parse_tick_data(
            data["tick"])
        self.tick_event(time, symbol, bid, ask) 
    
    def tick_event(self, time, symbol, bid, ask):
        midprice = (ask+bid)/2.
        self.prices.loc[time, symbol] = midprice

        resampled_prices = self.prices.resample(
            self.resample_interval,
            how='last',
            fill_method="ffill")

        mean_short = resampled_prices.tail(
            self.mean_period_short).mean()[0]
        mean_long = resampled_prices.tail(
            self.mean_period_long).mean()[0]
        self.beta = mean_short / mean_long

        self.perform_trade_logic(self.beta)
        self.calculate_unrealized_pnl(bid, ask)
        self.print_status()
        
    def perform_trade_logic(self, beta):
        if beta > self.buy_threshold:
            if not self.is_position_opened \
                    or self.position < 0:
                self.check_and_send_order(True)

        elif beta < self.sell_threshold:
            if not self.is_position_opened \
                    or self.position > 0:
                self.check_and_send_order(False)
    def print_status(self):
        print ("[%s] %s pos=%s beta=%s RPnL=%s UPnL=%s" % (
            datetime.now().time(),
            self.instrument, 
            self.position, 
            round(self.beta, 5),
            self.realized_pnl, 
            self.unrealized_pnl)
               )

In [ ]:
key = "04e86c6b0c7bfe71bf7f98d2b1a6496d-dee13f6f97c4508b949c42deaf95925b"
account_id = "101-001-6722193-001"
system = ForexSystem(environment="practice", access_token=key)
system.begin(accountId=account_id,
             instruments="EUR_USD",
             qty=1000,
             resample_interval="10s",
             mean_period_short=5,
             mean_period_long=20,
             buy_threshold=1.0,
             sell_threshold=1.0) 

In [4]:
import json
import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
accountID = "101-001-6722193-001"
api = API(access_token="04e86c6b0c7bfe71bf7f98d2b1a6496d-dee13f6f97c4508b949c42deaf95925b")
params ={"instruments": "EUR_USD,EUR_JPY"}
r = pricing.PricingStream(accountID,params=params)
rv = api.request(r)

for tick in rv:
    #print(json.dumps(ticks,indent=4),",")
    if(tick['type'] == 'PRICE'):
        print(tick) 

{'instrument': 'EUR_USD', 'bids': [{'price': '1.19635', 'liquidity': 10000000}], 'closeoutBid': '1.19620', 'status': 'tradeable', 'asks': [{'price': '1.19647', 'liquidity': 10000000}], 'tradeable': True, 'type': 'PRICE', 'time': '2017-09-22T15:26:08.256757252Z', 'closeoutAsk': '1.19662'}
{'instrument': 'EUR_JPY', 'bids': [{'price': '134.026', 'liquidity': 10000000}], 'closeoutBid': '134.026', 'status': 'tradeable', 'asks': [{'price': '134.042', 'liquidity': 10000000}], 'tradeable': True, 'type': 'PRICE', 'time': '2017-09-22T15:26:07.278679939Z', 'closeoutAsk': '134.042'}
{'instrument': 'EUR_USD', 'bids': [{'price': '1.19641', 'liquidity': 10000000}], 'closeoutBid': '1.19626', 'status': 'tradeable', 'asks': [{'price': '1.19651', 'liquidity': 10000000}], 'tradeable': True, 'type': 'PRICE', 'time': '2017-09-22T15:26:10.052967363Z', 'closeoutAsk': '1.19666'}
{'instrument': 'EUR_JPY', 'bids': [{'price': '134.030', 'liquidity': 10000000}], 'closeoutBid': '134.030', 'status': 'tradeable', 'as

KeyboardInterrupt: 

In [8]:
import oandapyV20
import pandas as pd
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
accountID = "101-001-6722193-001"
api = API(access_token="04e86c6b0c7bfe71bf7f98d2b1a6496d-dee13f6f97c4508b949c42deaf95925b")
params ={"instruments": "EUR_USD,EUR_JPY"}
r = pricing.PricingInfo(accountID,params=params)
rv = api.request(r)
pd.DataFrame(rv['prices'])

,asks,bids,closeoutAsk,closeoutBid,instrument,quoteHomeConversionFactors,status,time,tradeable,type,unitsAvailable
0,"[{'price': '1.19457', 'liquidity': 10000000}]","[{'price': '1.19444', 'liquidity': 10000000}]",1.19472,1.19429,EUR_USD,"{'positiveUnits': '1.00000000', 'negativeUnits...",tradeable,2017-09-22T20:21:56.235007019Z,True,PRICE,"{'openOnly': {'short': '4183513', 'long': '418..."
1,"[{'price': '133.878', 'liquidity': 10000000}]","[{'price': '133.855', 'liquidity': 10000000}]",133.878,133.855,EUR_JPY,"{'positiveUnits': '0.00892284', 'negativeUnits...",tradeable,2017-09-22T20:21:48.296013821Z,True,PRICE,"{'openOnly': {'short': '2091886', 'long': '209..."


In [23]:
import pandas as pd
intervals = '10s'
prices = pd.DataFrame()
time = pd.to_datetime('2017-09-25T17:57:41.646378854Z')
instrument = 'EUR_USD'
midprice = 1.18
prices.loc[time,instrument] = midprice
#prices.index = pd.to_datetime(prices.index, unit='s')
prices = prices.resample(intervals).last()
prices

,EUR_USD
2017-09-25 17:57:40,1.18
